In [1]:
import findspark
findspark.init()

Intro


https://github.com/databricks/Spark-The-Definitive-Guide/tree/master/data
https://github.com/VolodymyrGavrysh/My_RoadMap_Data_Science/blob/master/kds/books/Spark-The%20Definitive%20Guide.pdf


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext('local','test')

spark = SparkSession.builder.master('local').appName('test').getOrCreate()

# while creating sparksession or context, master and appname are mandatory 

toDF

In [4]:
data = spark.range(100).toDF('Number')
data.collect()

[Row(Number=0),
 Row(Number=1),
 Row(Number=2),
 Row(Number=3),
 Row(Number=4),
 Row(Number=5),
 Row(Number=6),
 Row(Number=7),
 Row(Number=8),
 Row(Number=9),
 Row(Number=10),
 Row(Number=11),
 Row(Number=12),
 Row(Number=13),
 Row(Number=14),
 Row(Number=15),
 Row(Number=16),
 Row(Number=17),
 Row(Number=18),
 Row(Number=19),
 Row(Number=20),
 Row(Number=21),
 Row(Number=22),
 Row(Number=23),
 Row(Number=24),
 Row(Number=25),
 Row(Number=26),
 Row(Number=27),
 Row(Number=28),
 Row(Number=29),
 Row(Number=30),
 Row(Number=31),
 Row(Number=32),
 Row(Number=33),
 Row(Number=34),
 Row(Number=35),
 Row(Number=36),
 Row(Number=37),
 Row(Number=38),
 Row(Number=39),
 Row(Number=40),
 Row(Number=41),
 Row(Number=42),
 Row(Number=43),
 Row(Number=44),
 Row(Number=45),
 Row(Number=46),
 Row(Number=47),
 Row(Number=48),
 Row(Number=49),
 Row(Number=50),
 Row(Number=51),
 Row(Number=52),
 Row(Number=53),
 Row(Number=54),
 Row(Number=55),
 Row(Number=56),
 Row(Number=57),
 Row(Number=58),
 Row(Nu

In [5]:
data.show()

+------+
|Number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows



In [6]:
sc.uiWebUrl

'http://localhost:4040'

where

In [7]:
even_data = data.where('Number % 2 = 0')
even_data.show()

+------+
|Number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



In [9]:
even_data = data.filter(even_data.Number%2 == 0)
even_data.show()

+------+
|Number|
+------+
|     0|
|     2|
|     4|
|     6|
|     8|
|    10|
|    12|
|    14|
|    16|
|    18|
|    20|
|    22|
|    24|
|    26|
|    28|
|    30|
|    32|
|    34|
|    36|
|    38|
+------+
only showing top 20 rows



predicate pushdown on DataFrames
If we build a large Spark job but specify a filter at the end that only requires us to
fetch one row from our source data, the most efficient way to execute this is to access the single
record that we need. Spark will actually optimize this for us by pushing the filter down
automatically.

There are 3 types of actions:

Actions to view data in the console
Actions to collect data to native objects in the respective language
Actions to write to output data sources

In [10]:
Summary_2015 = spark.read.option("inferSchema","True")\
.option('header','True')\
.csv('C:/Users/varsh/Downloads/2015-summary.csv')

take

In [11]:
Summary_2015.take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)]

In [13]:
Summary_2015.show(truncate=False)

+------------------------+-------------------+-----+
|DEST_COUNTRY_NAME       |ORIGIN_COUNTRY_NAME|count|
+------------------------+-------------------+-----+
|United States           |Romania            |15   |
|United States           |Croatia            |1    |
|United States           |Ireland            |344  |
|Egypt                   |United States      |15   |
|United States           |India              |62   |
|United States           |Singapore          |1    |
|United States           |Grenada            |62   |
|Costa Rica              |United States      |588  |
|Senegal                 |United States      |40   |
|Moldova                 |United States      |1    |
|United States           |Sint Maarten       |325  |
|United States           |Marshall Islands   |39   |
|Guyana                  |United States      |64   |
|Malta                   |United States      |1    |
|Anguilla                |United States      |41   |
|Bolivia                 |United States      |

explain

In [14]:
Summary_2015.sort('count').explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#43 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#43 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=125]
      +- FileScan csv [DEST_COUNTRY_NAME#41,ORIGIN_COUNTRY_NAME#42,count#43] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/varsh/Downloads/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [15]:
# By default, when we perform a shuffle, Spark
# outputs 200 shuffle partitions. Let’s set this value to 5 to reduce the number of the output
# partitions from the shuffle:

spark.conf.set("spark.sql.shuffle.partitions", "5")

Summary_2015.sort('count').take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Singapore', count=1),
 Row(DEST_COUNTRY_NAME='Moldova', ORIGIN_COUNTRY_NAME='United States', count=1)]

createOrReplaceTempView

In [16]:
Summary_2015.createOrReplaceTempView('data')

spark.sql

In [17]:
spark.sql('select * from data').show(truncate=False)

+------------------------+-------------------+-----+
|DEST_COUNTRY_NAME       |ORIGIN_COUNTRY_NAME|count|
+------------------------+-------------------+-----+
|United States           |Romania            |15   |
|United States           |Croatia            |1    |
|United States           |Ireland            |344  |
|Egypt                   |United States      |15   |
|United States           |India              |62   |
|United States           |Singapore          |1    |
|United States           |Grenada            |62   |
|Costa Rica              |United States      |588  |
|Senegal                 |United States      |40   |
|Moldova                 |United States      |1    |
|United States           |Sint Maarten       |325  |
|United States           |Marshall Islands   |39   |
|Guyana                  |United States      |64   |
|Malta                   |United States      |1    |
|Anguilla                |United States      |41   |
|Bolivia                 |United States      |

You can express your business logic in SQL or DataFrames (either in R, Python, Scala, or Java) and Spark will compile that logic down to an underlying plan (that you can see in the explain plan) before actually executing your code. With Spark SQL, you can register any DataFrame as a table or
view (a temporary table) and query it using pure SQL. There is no performance difference
between writing SQL queries or writing DataFrame code, they both “compile” to the same
underlying plan that we specify in DataFrame code.

In [31]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM data
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = Summary_2015\
.groupBy("DEST_COUNTRY_NAME")\
.count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#44], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#44, 5), ENSURE_REQUIREMENTS, [plan_id=127]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#44], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#44] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/varsh/Downloads/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#44], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#44, 5), ENSURE_REQUIREMENTS, [plan_id=140]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#44], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#44] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[fi

spark sql functions

In [18]:
from pyspark.sql.functions import max

Summary_2015.select(max("count")).take(1)

[Row(max(count)=370002)]

In [22]:
maxCount = Summary_2015.select(max("count"))

maxCount.write.parquet('C:/Users/varsh/outputs/maxCount')

In [24]:
CountMax = spark.read.format('parquet').load('C:/Users/varsh/outputs/maxCount')
CountMax.show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [33]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM data
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")
maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



cache

In [35]:
spark.table("data").cache()

# This returns the table as a dataframe and caches it. Caching also done post an action call

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: int]

In [50]:
# spark.sql('select * from data').show()

spark.table("data").take(2)

[Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15),
 Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Croatia', count=1)]

df.show() : It will show only the content of the dataframe.
df.show(n) 
df.collect() : It will show the content and metadata of the dataframe.
df.take(n) : shows content and structure/metadata for a limited number of rows for a very large dataset.

In [52]:
from pyspark.sql.functions import desc

Summary_2015\
.groupBy("DEST_COUNTRY_NAME")\
.sum("count")\
.withColumnRenamed("sum(count)", "destination_total")\
.sort(desc("destination_total"))\
.limit(5)\
.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



Triple quotes allow line breaks

In [ ]:
spark.sql('''select DEST_COUNTRY_NAME, sum(count) as destination_total from data
          group by 1 order by destination_total desc limit 5''').show()



+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



In [26]:
# Summary_2015.sort(desc('DEST_COUNTRY_NAME')).show(3)

Summary_2015.sort(Summary_2015.DEST_COUNTRY_NAME,ascending=False).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|           Zambia|      United States|    1|
|        Venezuela|      United States|  290|
+-----------------+-------------------+-----+
only showing top 2 rows



Chapter 3 -- To be continued -- page 39

In [28]:
retail = spark.read.option('InferSchema','True').option('Header','True').csv("C:/Users/varsh/Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")

In [29]:
retail.show(2)

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|        Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
|   580538|    23084| RABBIT NIGHT LIGHT|      48|2011-12-05 08:38:00|     1.79|   14075.0|United Kingdom|
|   580538|    23077|DOUGHNUT LIP GLOSS |      20|2011-12-05 08:38:00|     1.25|   14075.0|United Kingdom|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+
only showing top 2 rows



In [30]:
retail.createOrReplaceTempView("retail_table")
retail_schema = retail.schema

In [31]:
print(retail_schema)

StructType([StructField('InvoiceNo', StringType(), True), StructField('StockCode', StringType(), True), StructField('Description', StringType(), True), StructField('Quantity', IntegerType(), True), StructField('InvoiceDate', TimestampType(), True), StructField('UnitPrice', DoubleType(), True), StructField('CustomerID', DoubleType(), True), StructField('Country', StringType(), True)])


In [96]:
from pyspark.sql.functions import window, column, desc, col

# spark.sql('''select CustomerID, InvoiceDate, sum(UnitPrice * Quantity)
#            from retail_Table group by CustomerID,InvoiceDate
#           order by sum(UnitPrice * Quantity)''').show(truncate=False)

spark.sql('''select CustomerID, Date(InvoiceDate), DATE_ADD(InvoiceDate, 1),sum(UnitPrice * Quantity)
           from retail_Table group by CustomerID,Date(InvoiceDate), DATE_ADD(InvoiceDate, 1)
          order by sum(UnitPrice * Quantity)''').show(truncate=False)


+----------+-----------+------------------------+---------------------------+
|CustomerID|InvoiceDate|date_add(InvoiceDate, 1)|sum((UnitPrice * Quantity))|
+----------+-----------+------------------------+---------------------------+
|null      |2011-01-05 |2011-01-06              |-29716.63000000005         |
|null      |2011-05-16 |2011-05-17              |-10387.250000000062        |
|null      |2011-02-21 |2011-02-22              |-8437.31000000004          |
|null      |2010-12-07 |2010-12-08              |-8066.940000000085         |
|null      |2011-08-12 |2011-08-13              |-7862.940000000068         |
|null      |2011-07-13 |2011-07-14              |-7672.000000000031         |
|12931.0   |2011-08-31 |2011-09-01              |-7460.950000000001         |
|null      |2011-06-24 |2011-06-25              |-7367.370000000025         |
|16029.0   |2011-05-03 |2011-05-04              |-6930.0                    |
|null      |2011-09-15 |2011-09-16              |-6724.830000000

selectExpr

In [85]:
output = retail\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

output.sort("sum(total_cost)").show(truncate=False)



+----------+------------------------------------------+-------------------+
|CustomerId|window                                    |sum(total_cost)    |
+----------+------------------------------------------+-------------------+
|null      |{2011-01-05 05:30:00, 2011-01-06 05:30:00}|-29716.63000000005 |
|null      |{2011-05-16 05:30:00, 2011-05-17 05:30:00}|-10387.250000000062|
|null      |{2011-02-21 05:30:00, 2011-02-22 05:30:00}|-8437.31000000004  |
|null      |{2010-12-07 05:30:00, 2010-12-08 05:30:00}|-8066.940000000085 |
|null      |{2011-08-12 05:30:00, 2011-08-13 05:30:00}|-7862.940000000068 |
|null      |{2011-07-13 05:30:00, 2011-07-14 05:30:00}|-7672.000000000031 |
|12931.0   |{2011-08-31 05:30:00, 2011-09-01 05:30:00}|-7460.950000000001 |
|null      |{2011-06-24 05:30:00, 2011-06-25 05:30:00}|-7367.370000000025 |
|16029.0   |{2011-05-03 05:30:00, 2011-05-04 05:30:00}|-6930.0            |
|null      |{2011-09-15 05:30:00, 2011-09-16 05:30:00}|-6724.830000000022 |
|null      |

Spark Streaming

In [97]:
streamingDataFrame = spark.readStream\
.schema(retail_schema)\
.option("maxFilesPerTrigger", 1)\
.format("csv")\
.option("header", "true")\
.load("C:/Users/varsh/Spark-The-Definitive-Guide/data/retail-data/by-day/*.csv")

In [98]:
streamingDataFrame.isStreaming

True

In [99]:
purchaseByCustomerPerHour = streamingDataFrame\
.selectExpr(
"CustomerId",
"(UnitPrice * Quantity) as total_cost",
"InvoiceDate")\
.groupBy(
col("CustomerId"), window(col("InvoiceDate"), "1 day"))\
.sum("total_cost")

In [101]:
purchaseByCustomerPerHour.explain()

== Physical Plan ==
*(4) HashAggregate(keys=[CustomerId#2353, window#2378], functions=[sum(total_cost#2363)])
+- StateStoreSave [CustomerId#2353, window#2378], state info [ checkpoint = <unknown>, runId = c830aca3-7c6e-400d-b854-edcd3b46413d, opId = 0, ver = 0, numPartitions = 5], Append, 0, 2
   +- *(3) HashAggregate(keys=[CustomerId#2353, window#2378], functions=[merge_sum(total_cost#2363)])
      +- StateStoreRestore [CustomerId#2353, window#2378], state info [ checkpoint = <unknown>, runId = c830aca3-7c6e-400d-b854-edcd3b46413d, opId = 0, ver = 0, numPartitions = 5], 2
         +- *(2) HashAggregate(keys=[CustomerId#2353, window#2378], functions=[merge_sum(total_cost#2363)])
            +- Exchange hashpartitioning(CustomerId#2353, window#2378, 5), ENSURE_REQUIREMENTS, [plan_id=1726]
               +- *(1) HashAggregate(keys=[knownfloatingpointnormalized(normalizenanandzero(CustomerId#2353)) AS CustomerId#2353, window#2378], functions=[partial_sum(total_cost#2363)])
               

In [102]:
purchaseByCustomerPerHour.writeStream\
.format("memory")\
.queryName("customer_purchases")\
.outputMode("complete")\
.start()


Spark Streaming VS Structured Streaming

Notice how this window is built on
event time, as well, not the time at which Spark processes the data. This was one of the
shortcomings of Spark Streaming that Structured Streaming has resolved

Chapter 4 Structured API - Page 55

DataFrames Versus Datasets
In essence, within the Structured APIs, there are two more APIs, the “untyped” DataFrames and
the “typed” Datasets. To say that DataFrames are untyped is aslightly inaccurate; they have
types, but Spark maintains them completely and only checks whether those types line up to those
specified in the schema at runtime. Datasets, on the other hand, check whether types conform to
the specification at compile time. Datasets are only available to Java Virtual Machine (JVM)–
based languages (Scala and Java)

In [106]:
spark.range(2).toDF('col1').show()

+----+
|col1|
+----+
|   0|
|   1|
+----+



In [ ]:
from pyspark.sql.types import *
b = ByteType()


ByteType()
ByteType()
IntegerType()
LongType()
FloatType()
DoubleType()
DecimalType()
StringType()
BinaryType()
BooleanType()
TimestampType()
DateType()
ArrayType -- list, tuple, or array -- ArrayType(elementType,[containsNull])
MapType -- dict -- MapType(keyType,valueType,[valueContainsNull]).
StructType -- StructType(fields). Note:fields is a list of StructFields.
StructField -- StructField(name,dataType, [nullable])


usercode > if code is valid > unresolved logical plan > ....

This plan is unresolved
because although your code might be valid, the tables or columns that it refers to might or might
not exist. Spark uses the catalog, a repository of all table and DataFrame information, to resolve
columns and tables in the analyzer.  The analyzer might reject the unresolved logical plan if the
required table or column name does not exist in the catalog. If the analyzer can resolve it, the
result is passed through the Catalyst Optimizer, a collection of rules that attempt to optimize the
logical plan by pushing down predicates or selections

Analysis using catalog > resolved logical plan > optimised logical plan > physical plan



The physical plan, often called a Spark plan, specifies how the logical plan will execute
on the cluster by generating different physical execution strategies and comparing them through
a cost model, An example of the cost comparison might be choosing
how to perform a given join by looking at the physical attributes of a given table (how big the
table is or how big its partitions are).

Chapter 5. Basic Structured Operations


In [111]:
spark.read.format('json')\
.load('C:/Users/varsh/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json').schema

StructType([StructField('DEST_COUNTRY_NAME', StringType(), True), StructField('ORIGIN_COUNTRY_NAME', StringType(), True), StructField('count', LongType(), True)])

In [112]:
flight_json = spark.read.format('json')\
.load('C:/Users/varsh/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json')
flight_json.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: long (nullable = true)



Deciding whether you need to define a schema prior to reading in your data depends on your use case.
For ad hoc analysis, schema-on-read usually works just fine (although at times it can be a bit slow with
plain-text file formats like CSV or JSON). However, this can also lead to precision issues like a long
type incorrectly set as an integer when reading in a file. When using Spark for production Extract,
Transform, and Load (ETL), it is often a good idea to define your schemas manually, especially when
working with untyped data sources like CSV and JSON because schema inference can vary depending
on the type of data that you read in.


Defining Schema

In [122]:
from pyspark.sql.types import StructField, StructType, StringType, LongType

myManualSchema = StructType([
StructField("DEST_COUNTRY_NAME", StringType(), True),
StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
StructField("count", LongType(), False, metadata={"hello":"world"})
])

df = spark.read.format("json").schema(myManualSchema)\
.load("C:/Users/varsh/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json")

In [117]:
from pyspark.sql.functions import col, column

# col("someColumnName")
# column("someColumnName")

We mentioned earlier that columns are expressions, but what is an expression? An expression is
a set of transformations on one or more values in a record in a DataFrame. Think of it like a
function that takes as input one or more column names, resolves them, and then potentially
applies more expressions to create a single value for each record in the dataset. Importantly, this
“single value” can actually be a complex type like a Map or Array. We’ll see more of the
complex types in Chapter 6.
In the simplest case, an expression, created via the expr function, is just a DataFrame column
reference. In the simplest case, expr("someCol") is equivalent to col("someCol").

Spark SQL function - expr

In [118]:
from pyspark.sql.functions import expr
expr("(((someCol + 5) * 200) - 6) < otherCol")

Column<'((((someCol + 5) * 200) - 6) < otherCol)'>

In [120]:
spark.read.format("json").schema(myManualSchema)\
.load("C:/Users/varsh/Spark-The-Definitive-Guide/data/flight-data/json/2015-summary.json").columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count']

In [123]:
df.first()

Row(DEST_COUNTRY_NAME='United States', ORIGIN_COUNTRY_NAME='Romania', count=15)

In [9]:
from pyspark.sql import Row
myRow = Row("Hello", None, 1, False)

DataFrame Transformations

We can add rows or columns
We can remove rows or columns
We can transform a row into a column (or vice versa)
We can change the order of rows based on the values in columns

In [127]:
df.createOrReplaceTempView('flight')

page 74

In [128]:
df.select("DEST_COUNTRY_NAME", "ORIGIN_COUNTRY_NAME").show(2)


+-----------------+-------------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|
+-----------------+-------------------+
|    United States|            Romania|
|    United States|            Croatia|
+-----------------+-------------------+
only showing top 2 rows



In [129]:
from pyspark.sql.functions import expr, col, column
df.select(
expr("DEST_COUNTRY_NAME"),
col("DEST_COUNTRY_NAME"),
column("DEST_COUNTRY_NAME"))\
.show(2)


+-----------------+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+-----------------+
|    United States|    United States|    United States|
|    United States|    United States|    United States|
+-----------------+-----------------+-----------------+
only showing top 2 rows



In [131]:
df.select(col("DEST_COUNTRY_NAME"), "DEST_COUNTRY_NAME").show(2)


+-----------------+-----------------+
|DEST_COUNTRY_NAME|DEST_COUNTRY_NAME|
+-----------------+-----------------+
|    United States|    United States|
|    United States|    United States|
+-----------------+-----------------+
only showing top 2 rows



In [132]:
df.select(expr("DEST_COUNTRY_NAME AS destination")).show(2)


+-------------+
|  destination|
+-------------+
|United States|
|United States|
+-------------+
only showing top 2 rows



In [133]:
df.select(expr("DEST_COUNTRY_NAME as destination").alias("DEST_COUNTRY_NAME"))\
.show(2)


+-----------------+
|DEST_COUNTRY_NAME|
+-----------------+
|    United States|
|    United States|
+-----------------+
only showing top 2 rows



In [134]:
df.selectExpr("DEST_COUNTRY_NAME as newColumnName", "DEST_COUNTRY_NAME").show(2)


+-------------+-----------------+
|newColumnName|DEST_COUNTRY_NAME|
+-------------+-----------------+
|United States|    United States|
|United States|    United States|
+-------------+-----------------+
only showing top 2 rows



In [135]:
df.selectExpr(
"*", # all original columns
"(DEST_COUNTRY_NAME = ORIGIN_COUNTRY_NAME) as withinCountry")\
.show(2)

+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [136]:
df.selectExpr("avg(count)", "count(distinct(DEST_COUNTRY_NAME))").show(2)


+-----------+---------------------------------+
| avg(count)|count(DISTINCT DEST_COUNTRY_NAME)|
+-----------+---------------------------------+
|1770.765625|                              132|
+-----------+---------------------------------+



String literals

Sometimes, we need to pass explicit values into Spark that are just a value (rather than a new
column). This might be a constant value or something we’ll need to compare to later on. The
way we do this is through literals

In [137]:
from pyspark.sql.functions import lit
df.select(expr("*"), lit(1).alias("One")).show(2)

+-----------------+-------------------+-----+---+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|One|
+-----------------+-------------------+-----+---+
|    United States|            Romania|   15|  1|
|    United States|            Croatia|    1|  1|
+-----------------+-------------------+-----+---+
only showing top 2 rows



Notice that the withColumn function takes two arguments: the column name and the expression
that will create the value for that given row in the DataFrame. Interestingly, we can also rename
a column this way

In [138]:
df.withColumn("withinCountry", expr("ORIGIN_COUNTRY_NAME == DEST_COUNTRY_NAME"))\
.show(2)


+-----------------+-------------------+-----+-------------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|withinCountry|
+-----------------+-------------------+-----+-------------+
|    United States|            Romania|   15|        false|
|    United States|            Croatia|    1|        false|
+-----------------+-------------------+-----+-------------+
only showing top 2 rows



In [139]:
df.withColumn("Destination", expr("DEST_COUNTRY_NAME")).columns


['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'Destination']

Although we can rename a column in the manner that we just described, another alternative is to
use the withColumnRenamed method.

In [140]:
df.withColumnRenamed("DEST_COUNTRY_NAME", "dest").columns


['dest', 'ORIGIN_COUNTRY_NAME', 'count']

Reserved character and escape character

In [141]:
dfWithLongColName = df.withColumn(
"This Long Column-Name",
expr("ORIGIN_COUNTRY_NAME"))

In [142]:
dfWithLongColName.columns

['DEST_COUNTRY_NAME', 'ORIGIN_COUNTRY_NAME', 'count', 'This Long Column-Name']

In [143]:
dfWithLongColName.selectExpr(
"`This Long Column-Name`",
"`This Long Column-Name` as `new col`")\
.show(2)

+---------------------+-------+
|This Long Column-Name|new col|
+---------------------+-------+
|              Romania|Romania|
|              Croatia|Croatia|
+---------------------+-------+
only showing top 2 rows



Here, we need to use backticks because we’re referencing a column in an expression:


In [151]:
dfWithLongColName.select(expr("`This Long Column-Name`")).columns


['This Long Column-Name']

In [148]:
dfWithLongColName.select(col("This Long Column-Name")).show(2)


+---------------------+
|This Long Column-Name|
+---------------------+
|              Romania|
|              Croatia|
+---------------------+
only showing top 2 rows



By default Spark is case insensitive; however, you can make Spark case sensitive by setting the
configuration:
-- in SQL
set spark.sql.caseSensitive true

We can drop multiple columns by passing in multiple columns as arguments:

In [153]:
dfWithLongColName.drop("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").columns


['count', 'This Long Column-Name']

Changing a Column’s Type (cast)

In [159]:
df.withColumn("count2", col("count").cast("String")).dtypes


[('DEST_COUNTRY_NAME', 'string'),
 ('ORIGIN_COUNTRY_NAME', 'string'),
 ('count', 'bigint'),
 ('count2', 'string')]

In [163]:
df.filter(col("count") < 2).show(2)
df.filter("count>2").show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|            Anguilla|       United States|   41|

In [164]:
df.where("count < 2").show(2)
df.where(col("count")>500).show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+------------------+-------------------+------+
| DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+------------------+-------------------+------+
|        Costa Rica|      United States|   588|
|     United States|        Netherlands|   660|
|       The Bahamas|      United States|   955|
|       El Salvador|      United States|   561|
|            Mexico|      United States|  7140|
|     United States|         Costa Rica|   608|
|          Colombia|      United States|   873|
|     United States|            Jamaica|   712|
|            Panama|      United States|   510|
|     United States|        The Bahamas|   986|
|     United States|              China|   920|
|     United States| Domini

distinct

In [166]:
df.select("ORIGIN_COUNTRY_NAME").distinct().count()


125

In [165]:
df.select("ORIGIN_COUNTRY_NAME", "DEST_COUNTRY_NAME").distinct().count()


256

Random Samples
Sometimes, you might just want to sample some random records from your DataFrame. You can
do this by using the sample method on a DataFrame, which makes it possible for you to specify
a fraction of rows to extract from a DataFrame and whether you’d like to sample with or without
replacement:

In [170]:
seed = 5
withReplacement = False
fraction = 0.5
df.sample(withReplacement, fraction, seed).show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Croatia|    1|
|       United States|               India|   62|
|       United States|           Singapore|    1|
|       United States|             Grenada|   62|
|             Senegal|       United States|   40|
|             Moldova|       United States|    1|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|             Bolivia|       United States|   30|
|       United States|            Paraguay|    6|
|       United States|           Gibraltar|    1|
|Saint Vincent and...|       United States|    1|
|               Italy|       United States|  382|
|       United States|Federated States ...|   69|
|       United States|              Russia|  161|
|       United States|         Netherlands|  660|


In [171]:
dataFrames = df.randomSplit([0.25, 0.75], seed)
dataFrames[0].count() > dataFrames[1].count()

False

In [172]:
print(dataFrames)

[DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint], DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]]


Concatenating and Appending Rows (Union)

page - 84

In [176]:
from pyspark.sql import Row
schema = df.schema
newRows = [
Row("New Country", "Other Country", 5),
Row("New Country 2", "Other Country 3", 1)
]
parallelizedRows = spark.sparkContext.parallelize(newRows)
newDF = spark.createDataFrame(parallelizedRows, schema)


In [177]:
df.union(newDF)\
.where("count = 1")\
.where(col("ORIGIN_COUNTRY_NAME") != "United States")\
.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Croatia|    1|
|    United States|          Singapore|    1|
|    United States|          Gibraltar|    1|
|    United States|             Cyprus|    1|
|    United States|            Estonia|    1|
|    United States|          Lithuania|    1|
|    United States|           Bulgaria|    1|
|    United States|            Georgia|    1|
|    United States|            Bahrain|    1|
|    United States|   Papua New Guinea|    1|
|    United States|         Montenegro|    1|
|    United States|            Namibia|    1|
|    New Country 2|    Other Country 3|    1|
+-----------------+-------------------+-----+



In [178]:
df.sort("count").show(5)
df.orderBy("count", "DEST_COUNTRY_NAME").show(5)
df.orderBy(col("count"), col("DEST_COUNTRY_NAME")).show(5)

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|               Malta|      United States|    1|
|Saint Vincent and...|      United States|    1|
|       United States|            Croatia|    1|
|       United States|          Gibraltar|    1|
|       United States|          Singapore|    1|
+--------------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|     Burkina Faso|      United States|    1|
|    Cote d'Ivoire|      United States|    1|
|           Cyprus|      United States|    1|
|         Djibouti|      United States|    1|
|        Indonesia|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--

In [179]:
from pyspark.sql.functions import desc, asc
df.orderBy(expr("count desc")).show(2)
df.orderBy(col("count").desc(), col("DEST_COUNTRY_NAME").asc()).show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|          Moldova|      United States|    1|
|    United States|            Croatia|    1|
+-----------------+-------------------+-----+
only showing top 2 rows

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|    United States|             Canada|  8483|
+-----------------+-------------------+------+
only showing top 2 rows



sortWithinPartitions

In [180]:
data = spark.read.format("json").load("C:/Users/varsh/Spark-The-Definitive-Guide/data/flight-data/json/*-summary.json")\
.sortWithinPartitions("count")

limit

In [182]:
df.limit(5).show()


+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



getNumPartitions

In [183]:
df.rdd.getNumPartitions()

1

In [184]:
data.rdd.getNumPartitions()

1

repartition

In [185]:
df.repartition(col("DEST_COUNTRY_NAME"))

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]

In [187]:
df.repartition(5, col("DEST_COUNTRY_NAME")).coalesce(2)

DataFrame[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string, count: bigint]